# `.groupby()` and `.agg()` - Part 2

The purpose of this tutorial is to demonstrate how to use `.groupby()` and `.agg()` with user defined functions.

In service of this objective, our analysis goal will be to calculate three monthly statistics for several ETFs during the year of 2020.  The three statistics we will calculate are:

1. average daily volume

2. monthly return

3. monthly volatility

### Loading Packages

Let's load the packages that we will need.

In [1]:
import numpy as np
import pandas as pd
import pandas_datareader as pdr

### Reading-In Data

In this tutorial we will be working with price data for the Select Sector SPDR ETFs.  Each of these funds tracks a particular subset (sector) of the SP&500 Index.  For example, XLF tracks the financial sector and has major holdings in JP Morgan, Wells Fargo, and Bank of America.

Let grab the data from Yahoo:

In [2]:
pd.options.display.max_rows = 25
lst_symbols = ['XLY', 'XLP', 'XLE', 'XLF', 'XLV', 'XLI', 'XLB', 'XLRE', 'XLK', 'XLU',]
df_etf = pdr.get_data_yahoo(lst_symbols, start='2020-01-01', end='2020-12-31')
df_etf = df_etf.round(2)
df_etf.head()

Attributes Adj Close                                                          \
Symbols          XLY    XLP    XLE    XLF    XLV    XLI    XLB   XLRE    XLK   
Date                                                                           
2020-01-02    125.12  60.05  55.63  30.04  99.66  80.85  58.88  36.34  91.81   
2020-01-03    124.05  59.96  55.47  29.72  98.79  80.69  57.93  36.61  90.78   
2020-01-06    124.40  60.08  55.90  29.70  99.40  80.72  57.68  36.62  90.99   
2020-01-07    124.20  59.62  55.75  29.51  99.21  80.55  57.61  36.22  90.95   
2020-01-08    124.58  59.84  54.83  29.70  99.85  80.83  57.82  36.40  91.93   

Attributes         ...     Volume                                      \
Symbols       XLU  ...        XLY         XLP         XLE         XLF   
Date               ...                                                  
2020-01-02  60.67  ...  6295500.0  14460700.0  11944700.0  28843300.0   
2020-01-03  60.80  ...  5596400.0  26388900.0  29502900.0  51363600.0   
2020-01-06  60.85  ...  6411600.0  22541700.0  22458100.0  27956100.0   
2020-01-07  60.77  ...  9150800.0  15607600.0  11462500.0  39627500.0   
2020-01-08  60.74  ...  4725900.0  11451400.0  19021400.0  47966600.0   

Attributes                                                            \
Symbols           XLV         XLI         XLB       XLRE         XLK   
Date                                                                   
2020-01-02  6277400.0  16121300.0   7357400.0  4380100.0  13283500.0   
2020-01-03  8247500.0  17571300.0  12423200.0  3499000.0  15011800.0   
2020-01-06  6441800.0  16153100.0  15764400.0  3097200.0   7815000.0   
2020-01-07  6335300.0  16675400.0  20266900.0  3550600.0   7681800.0   
2020-01-08  7494700.0  10677700.0   8079600.0  5089000.0  11627200.0   

Attributes              
Symbols            XLU  
Date                    
2020-01-02  19107700.0  
2020-01-03  17989300.0  
2020-01-06  10444500.0  
2020-01-07  13070300.0  
2020-01-08  12741400.0  

[5 rows x 60 columns]

This data is not as tidy as we would like.  Let's use method chaining to perform a series of data munging operations.

In [3]:
df_etf = \
    (
    df_etf
        .stack(level='Symbols') #pivot the table
        .reset_index() #turn date into a column 
        .sort_values(by=['Symbols', 'Date']) #sort
        .rename(columns={'Date':'date', 'Symbols':'symbol', 'Adj Close':'adj_close','Close':'close', 
                         'High':'high', 'Low':'low', 'Open':'open', 'Volume':'volume'}) #renaming columns
        [['date', 'symbol','open', 'high', 'low', 'close', 'volume', 'adj_close']] #reordering columns
    )
df_etf

Attributes,date,symbol,open,high,low,close,volume,adj_close
6,2020-01-02,XLB,61.83,61.94,60.63,60.70,7357400.0,58.88
16,2020-01-03,XLB,60.08,60.44,59.70,59.72,12423200.0,57.93
26,2020-01-06,XLB,59.55,59.83,59.41,59.46,15764400.0,57.68
36,2020-01-07,XLB,59.36,59.80,59.20,59.39,20266900.0,57.61
46,2020-01-08,XLB,59.40,59.84,59.20,59.60,8079600.0,57.82
...,...,...,...,...,...,...,...,...
2480,2020-12-24,XLY,157.70,158.12,157.21,157.88,1048800.0,157.40
2490,2020-12-28,XLY,159.42,160.32,158.60,159.68,2912400.0,159.19
2500,2020-12-29,XLY,160.24,160.53,158.98,159.73,2431200.0,159.24
2510,2020-12-30,XLY,160.30,160.93,160.13,160.69,2440700.0,160.20


**Coding Challenge:** Use a `DataFrame` attribute to determine the number of rows and columns in `df_etf`.

In [4]:
df_etf.shape

(2530, 8)

### Exploring and Cleaning the Data

As we can see from the coding challenge, this data set is large (by our standards).  Whenever I encounter a new data set that I can't look at in its entirety, I like to do a bit of exploration via the built-in `pandas` methods.

We know we have a variety of ETFs in our data, but it would be useful to know how many (especially if we were expecting a certain number).

In [5]:
print(df_etf['symbol'].unique())
print(df_etf['symbol'].unique().size)

['XLB' 'XLE' 'XLF' 'XLI' 'XLK' 'XLP' 'XLRE' 'XLU' 'XLV' 'XLY']
10


**Coding Challenge:** What `DataFrame` attribute could we use to check the data types of the columns of `df_etf`?

In [6]:
df_etf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2530 entries, 6 to 2520
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2530 non-null   datetime64[ns]
 1   symbol     2530 non-null   object        
 2   open       2530 non-null   float64       
 3   high       2530 non-null   float64       
 4   low        2530 non-null   float64       
 5   close      2530 non-null   float64       
 6   volume     2530 non-null   float64       
 7   adj_close  2530 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 177.9+ KB


When I work with a time series of daily prices, I like to check the first and last trade dates that are represented in the data.

In [7]:
print(df_etf['date'].min())
print(df_etf['date'].max())

2020-01-02 00:00:00
2020-12-31 00:00:00


Here is what we know about our data set thus far:

1. 10 different ETFs are represented.

1. Prices are coming from the entirety of 2020.


Here are some things that we aren't necessarily sure of that would be worth checking in a high-stakes situation:

1. Is there a row/price for each symbol on each trade date?

1. Is there ever more than one row/price for a given symbol on a given trade date?

We won't bother answering these questions for the purposes of this tutorial, but these are the types of data-integrity questions I will often try to answer when encountering a new data set.

### Calculating Daily Returns with `groupby()`

Our ultimate goal is to calculate monthly returns and monthly volatilities for each ETF in `df_etf`.  These quantities are both functions of daily returns.  So, our first order of business is to calculate daily returns. 

In a previous tutorial we calculated daily returns in a simple vectorized fashion.  Unfortunately, we can't use the exact same approach here because there are multiple ETFs in the data set.

To overcome this challenge we will use our first application of `.groupby()`.

Here is the `.groupby()` code that calculates daily returns for each ETF.

In [8]:
# sorting values to get everything in the right order
df_etf.sort_values(['symbol', 'date'], inplace=True)

# vectorized return calculation
df_etf['dly_ret'] = \
    df_etf['close'].groupby(df_etf['symbol']).pct_change()
df_etf.head()

Attributes,date,symbol,open,high,low,close,volume,adj_close,dly_ret
6,2020-01-02,XLB,61.83,61.94,60.63,60.70,7357400.0,58.88,NaN
16,2020-01-03,XLB,60.08,60.44,59.70,59.72,12423200.0,57.93,-0.016145
26,2020-01-06,XLB,59.55,59.83,59.41,59.46,15764400.0,57.68,-0.004354
36,2020-01-07,XLB,59.36,59.80,59.20,59.39,20266900.0,57.61,-0.001177
46,2020-01-08,XLB,59.40,59.84,59.20,59.60,8079600.0,57.82,0.003536


### Adding `year` and `month` Columns

The ultimate goal is to calculate monthly statistics for each of the ETFs in our data set.

As a preliminary step, let's add a month and year column to the `df_etf` by utilizing the `.dt` attribute that `pandas` provides for date columns.

In [9]:
df_etf['year'] = df_etf['date'].dt.year
df_etf['month'] = df_etf['date'].dt.month
df_etf[['date', 'year', 'month']].head()

Attributes,date,year,month
6,2020-01-02,2020,1
16,2020-01-03,2020,1
26,2020-01-06,2020,1
36,2020-01-07,2020,1
46,2020-01-08,2020,1


Let's do a quick data-integrity check:  There are 10 ETFs in our data set and there are 12 months in a year, so the number of symbol-year-month combinations should be 120.

The following code counts the number of rows associated with each symbol-year-month combination and puts that data into a `DataFrame`.

In [10]:
df_num_rows = \
    df_etf.groupby(['symbol', 'year', 'month']).size().reset_index()
df_num_rows.head()

,symbol,year,month,0
0,XLB,2020,1,21
1,XLB,2020,2,19
2,XLB,2020,3,22
3,XLB,2020,4,21
4,XLB,2020,5,20


**Coding Challenge:** Confirm that there are the correct number of symbol-year-month combinations in `df_num_rows`.

In [11]:
df_num_rows.shape

(120, 4)

Now that we've added the `year` and `month` columns we can proceed to calculating our monthly statistics.

### Average Daily Volume

Let's start with the most straight-forward calculation: average daily volume, over each month, for all 10 of the ETFs in our data set.  

This amounts to:

1. grouping by `symbol`, `month`, and `year`

1. applying the built-in `np.mean()` function to the `volume` column 

In [12]:
df_volume = \
    df_etf.groupby(['symbol', 'year', 'month'])['volume'].agg([np.mean]).reset_index()
df_volume.rename(columns={'mean':'avg_volume'}, inplace=True)
df_volume.head()

,symbol,year,month,avg_volume
0,XLB,2020,1,7.235429e+06
1,XLB,2020,2,1.058022e+07
2,XLB,2020,3,1.432920e+07
3,XLB,2020,4,9.000557e+06
4,XLB,2020,5,4.829185e+06


**Coding Challenge:** Calculate the maximum daily volume for each symbol, *over the entire year*.

In [13]:
df_etf.groupby(['symbol', 'year'])['volume'].agg([np.max]).reset_index()

,symbol,year,amax
0,XLB,2020,30741700.0
1,XLE,2020,99356700.0
2,XLF,2020,256525000.0
3,XLI,2020,79118200.0
4,XLK,2020,61727100.0
5,XLP,2020,50978800.0
6,XLRE,2020,49899800.0
7,XLU,2020,90263100.0
8,XLV,2020,39561900.0
9,XLY,2020,20616100.0


### Monthly Returns

Next, let's calculate monthly returns for each of the ETFs in our data set.  This amounts to:

1. grouping by `symbol`, `month`, and `year`

1. applying an aggregation function to the `daily_returns` column

These are the same two steps that we have done in our previous aggregation examples.  However, there is one additional wrinkle that we are going to have to contend with. 


In the previous section, we used simple built-in aggregation funtions available through `numpy`, such as `np.max` and `np.mean`.  Calculating monthly returns from daily returns is a little more complicated.

Thus, we are going to have to first create a custom function for calculating monthly returns from daily returns, and then use this custom function in `.agg()`.

The following code defines our monthly returns function in terms of daily returns:

In [14]:
def monthly_ret(dly_ret):
    return np.prod(1 + dly_ret) - 1

Now we can apply `monthly_ret` for all of our ETFs using the following code.

In [15]:
df_ret = \
    df_etf.groupby(['symbol', 'month', 'year'])['dly_ret'].agg([monthly_ret]).reset_index()
df_ret.head()

,symbol,month,year,monthly_ret
0,XLB,1,2020,-0.050577
1,XLB,2,2020,-0.085199
2,XLB,3,2020,-0.145675
3,XLB,4,2020,0.151865
4,XLB,5,2020,0.068813


We can see from our calculation that in March of 2020 XLB had a monthly return of -14.6%.

**Coding Challenge:** Which ETF had the highest single monthly return in all of 2020?  What was the month?

In [16]:
df_ret.sort_values(by=['monthly_ret'], ascending=False)

,symbol,month,year,monthly_ret
15,XLE,4,2020,0.307639
22,XLE,11,2020,0.279944
111,XLY,4,2020,0.188825
34,XLF,11,2020,0.168483
46,XLI,11,2020,0.160274
...,...,...,...,...
74,XLRE,3,2020,-0.157380
20,XLE,9,2020,-0.159888
38,XLI,3,2020,-0.192529
26,XLF,3,2020,-0.216999


### Monthly Volatility

Let's use a similar process to calculate the monthly volatility for each of the ETFs.

We begin by defining a custom function that calculates the monthly volatility from daily returns.  Recall that industry convention is to state these volatilities in annualized terms.

In [17]:
def monthly_vol(dly_ret):
    return np.std(dly_ret) * np.sqrt(252)

We can now use our `monthly_vol` function in to perform an aggregating calculation.

In [18]:
df_vol = \
    df_etf.groupby(['symbol', 'month', 'year'])['dly_ret'].agg([monthly_vol]).reset_index()
df_vol.head()

,symbol,month,year,monthly_vol
0,XLB,1,2020,0.150336
1,XLB,2,2020,0.282201
2,XLB,3,2020,0.932265
3,XLB,4,2020,0.503394
4,XLB,5,2020,0.277311


**Coding Challenge:**  What was the volatility for XLF in December 2018?

In [19]:
df_vol.query('symbol == "XLF" & month == 12')

,symbol,month,year,monthly_vol
35,XLF,12,2020,0.138521


### Combining Metrics - `inner join`

Now, suppose that we want to combine our three metrics into one report - meaning that we want them organized into one `DataFrame` in an easy to read fashion.

One way to do this is to use the `pandas.merge()` method that we learned in the previous tutorial to join together `df_volume` (average daily volume), `df_ret` (monthly returns), and `df_vol` (monthly volatility). 

In [20]:
df_joined = \
    (
    df_volume
        .merge(df_ret, on=['symbol', 'year', 'month'])
        .merge(df_vol, on=['symbol', 'year', 'month'])
    )
df_joined.head()

,symbol,year,month,avg_volume,monthly_ret,monthly_vol
0,XLB,2020,1,7.235429e+06,-0.050577,0.150336
1,XLB,2020,2,1.058022e+07,-0.085199,0.282201
2,XLB,2020,3,1.432920e+07,-0.145675,0.932265
3,XLB,2020,4,9.000557e+06,0.151865,0.503394
4,XLB,2020,5,4.829185e+06,0.068813,0.277311


## Combining Metrics - multiple aggregation

Another way to combine all our statistics into a single `DataFrame` is to supply all of our custom aggregation fuctions as arguments to the `.agg()` function in one shot.  

Here is what that looks like:

In [21]:
# defining aggregations
agg_funcs = \
    {'volume':[np.mean], 'dly_ret':[monthly_ret, monthly_vol]}

# performing all aggregations all three aggregations at once
df_joined = \
    df_etf.groupby(['symbol', 'month', 'year']).agg(agg_funcs).reset_index()

# looking at the data frame
df_joined.head()

symbol month  year        volume     dly_ret            
                              mean monthly_ret monthly_vol
0    XLB     1  2020  7.235429e+06   -0.050577    0.150336
1    XLB     2  2020  1.058022e+07   -0.085199    0.282201
2    XLB     3  2020  1.432920e+07   -0.145675    0.932265
3    XLB     4  2020  9.000557e+06    0.151865    0.503394
4    XLB     5  2020  4.829185e+06    0.068813    0.277311

Notice that the input into the `.agg()` method is a `dict` whose elements are pairs that look like: 

`'column_name':[list_of_aggregating_functions`].

**Code Challenge:** Modify the code above to add maximum daily volume to the report.

In [22]:
# defining aggregations
agg_funcs = \
    {'volume':[np.mean, np.max], 'dly_ret':[monthly_ret, monthly_vol]}

# performing all aggregations all three aggregations at once
df_joined = \
    df_etf.groupby(['symbol', 'month', 'year']).agg(agg_funcs).reset_index()

# looking at the data frame
df_joined.head()

symbol month  year        volume                 dly_ret            
                              mean        amax monthly_ret monthly_vol
0    XLB     1  2020  7.235429e+06  20266900.0   -0.050577    0.150336
1    XLB     2  2020  1.058022e+07  30741700.0   -0.085199    0.282201
2    XLB     3  2020  1.432920e+07  28390200.0   -0.145675    0.932265
3    XLB     4  2020  9.000557e+06  30738000.0    0.151865    0.503394
4    XLB     5  2020  4.829185e+06   7386300.0    0.068813    0.277311

## Related Reading

*PDSH* - 3.7 - Combining Datasets: Merging and Joining

*PDSH* - 3.8 - Aggregation and Grouping

*Python for Data Analysis (McKinney)* - Chapter 9 (pp 251-274) Data Aggregation and Grouping Operations